# Unidad 04
## Pivoteando Datos

In [ ]:
#r "nuget: Microsoft.DotNet.Interactive.SqlServer, *-*"

In [ ]:
foreach (var line in System.IO.File.ReadLines(".env"))
{
    var parts = line.Split('=');
    if (parts.Length == 2)
    {
        var key = parts[0].Trim();
        var value = parts[1].Trim();
        Environment.SetEnvironmentVariable(key, value);
    }
}

string connectionString = "Server=" + Environment.GetEnvironmentVariable("Server") + ";Database=" + Environment.GetEnvironmentVariable("Database") + ";User Id=" + Environment.GetEnvironmentVariable("User") + ";Password=" + Environment.GetEnvironmentVariable("Password") + ";Encrypt=True;TrustServerCertificate=True;Connection Timeout=30;";


In [ ]:
#!connect mssql --kernel-name Pampero @csharp:connectionString

In [ ]:
USE Pampero;
DROP TABLE IF EXISTS dbo.Ordenes;
CREATE TABLE dbo.Ordenes
(
ordenid INT NOT NULL,
fechaorden DATE NOT NULL,
empid INT NOT NULL,
clienteid VARCHAR(5) NOT NULL,
cantidad INT NOT NULL,
CONSTRAINT PK_Orders PRIMARY KEY(ordenid)
);


In [ ]:
INSERT INTO dbo.Ordenes(ordenid, fechaorden, empid, clienteid, cantidad) VALUES
(30001, '20140802', 3, 'A', 10),
(10001, '20141224', 2, 'A', 12),
(10005, '20141224', 1, 'B', 20),
(40001, '20150109', 2, 'A', 40),
(10006, '20150118', 1, 'C', 14),
(20001, '20150212', 2, 'B', 12),
(40005, '20160212', 3, 'A', 10),
(20002, '20160216', 1, 'C', 20),
(30003, '20160418', 2, 'B', 15),
(30004, '20140418', 3, 'C', 22),
(30007, '20160907', 3, 'D', 30);
SELECT * FROM dbo.Ordenes;


In [ ]:
SELECT empid, clienteid, SUM(cantidad) AS sumqty
FROM dbo.Ordenes
GROUP BY empid, clienteid;

## Los necesitamos asi
|empid|A|B       |C|D|
|---------|--------|----------|----------|----------|
|1|NULL|20|34|NULL|
|2|52|27|NULL|NULL|
|3|20|NULL|2|30|


In [ ]:
SELECT empid,
SUM(CASE WHEN clienteid = 'A' THEN cantidad END) AS A,
SUM(CASE WHEN clienteid = 'B' THEN cantidad END) AS B,
SUM(CASE WHEN clienteid = 'C' THEN cantidad END) AS C,
SUM(CASE WHEN clienteid = 'D' THEN cantidad END) AS D
FROM dbo.Ordenes
GROUP BY empid;


In [ ]:
SELECT empid, A, B, C, D
FROM (SELECT empid, clienteid, cantidad
FROM dbo.Ordenes) AS D
PIVOT(SUM(cantidad) FOR clienteid IN(A, B, C, D)) AS P;


In [ ]:
SELECT clienteid, [1], [2], [3]
FROM (SELECT empid, clienteid, cantidad
FROM dbo.Ordenes) AS D
PIVOT(SUM(cantidad) FOR empid IN([1], [2], [3])) AS P;


In [ ]:
DECLARE @colnameList varchar(200)
SET @colnameList = NULL
SELECT @colnameList = COALESCE(@colnameList + ',','') + clienteid
FROM (SELECT DISTINCT clienteid FROM dbo.Ordenes) AS O;
DECLARE @SQLQuery NVARCHAR(MAX);
SET @SQLQuery =
'SELECT empid, ' + @colnameList + '
FROM (SELECT empid, clienteid, cantidad
FROM dbo.Ordenes) AS D
PIVOT(SUM(cantidad) FOR clienteid IN(' + @colnameList + ')) AS P'

EXEC(@SQLQuery)


## Despivoteando

In [ ]:
USE Pampero;
DROP TABLE IF EXISTS dbo.EmpClieOrdenes;
CREATE TABLE dbo.EmpClieOrdenes
(
empid INT NOT NULL
CONSTRAINT PK_EmpClieOrdenes PRIMARY KEY,
A VARCHAR(5) NULL,
B VARCHAR(5) NULL,
C VARCHAR(5) NULL,
D VARCHAR(5) NULL
);

In [ ]:
INSERT INTO dbo.EmpClieOrdenes(empid, A, B, C, D)
SELECT empid, A, B, C, D
FROM (SELECT empid, clienteid, cantidad
FROM dbo.Ordenes) AS D
PIVOT(SUM(cantidad) FOR clienteid IN(A, B, C, D)) AS P;
SELECT * FROM dbo.EmpClieOrdenes;

### Producción de copias

In [ ]:
SELECT *
FROM dbo.EmpClieOrdenes
CROSS JOIN (VALUES('A'),('B'),('C'),('D')) AS C(clienteid);

### Extracción de valores

In [ ]:
SELECT empid, clienteid, cantidad
FROM dbo.EmpClieOrdenes
CROSS APPLY (VALUES('A', A),('B', B),('C', C),('D', D)) AS C(clienteid, cantidad);

### Eliminación de filas irrelevantes

In [ ]:
SELECT empid, clienteid, cantidad
FROM dbo.EmpClieOrdenes
CROSS APPLY (VALUES('A', A),('B', B),('C', C),('D', D)) AS C(clienteid, cantidad)
WHERE cantidad IS NOT NULL;

### UNPIVOT

In [ ]:
SELECT empid, clienteid, cantidad
FROM dbo.EmpClieOrdenes
    UNPIVOT(cantidad FOR clienteid IN (A, B, C, D)) AS U;